In [1]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from io import StringIO 

import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras

In [2]:
#Postgres credentials
pw = 'npF5SlOzvW'
db = "d2b_assessment" 
user='byroneji4734'
host='d2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com'
port = '5432'


#s3 credentials
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

bucket_name =  "d2b-internal-assessment-bucket"
Prefix='orders_data/'
res = s3.list_objects_v2(
    Bucket=bucket_name,
    Prefix=Prefix)

In [5]:
def create_server_connection():
    connection = None
    try:

        connection = psycopg2.connect(
            database=db,
            user=user,
            password=pw,
            host=host,
            port=port
        )
        
        print("PostgreSql Database connection successful")
    except:
        print("Error")

    return connection


connection = create_server_connection()

PostgreSql Database connection successful


In [ ]:
def read_and_convert_excel_to_dataframe(df_name,file_path):

    obj = s3.get_object(Bucket= bucket_name, Key= Prefix+file_path)
    df_name = pd.read_excel(obj['Body']) 
    df_name = pd.DataFrame(df_name)
    for column in df_name.columns:
    df_name = df_name.withColumnRenamed(column, '_'.join(column.split()).lower())
    df_name['client_name'] = df_name.client_name.to_title()
    
    return df_name 


read_and_convert_excel_to_dataframe('maxwell','maxwell.xlsx') 

In [ ]:
def load_df_to_postgres(conn, df, table):

    tuples = [list(row) for row in df.itertuples(index=False)]


    cols = ','.join(list(df.columns))
    
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)                   
    cursor = conn.cursor()
    
    try:
        extras.execute_values(cursor, query, tuples)
        #cursor.execute(query)
        conn.commit()
    
    except (Exception, psycopg2.DatabaseError):
        print("Error: %s")
        conn.rollback()
        cursor.close()
        return 1
    except psycopg2.InterfaceError:
        pass
    print("load to postgresql successful")
    cursor.close()

    
    
    
    
load_df_to_postgres(connection, orders, 'byroneji4734_staging.orders')